In [4]:
from pymongo import MongoClient
import pandas as pd
from bson import ObjectId
from functions.database import officesDataFrame
client = MongoClient()
db = client['companies']

In [9]:
#Create dataframe containing all the offices of the startup_tech companies
#Create dataframe containing all the offices available for renting

df_startups = officesDataFrame(db,{'isTech':True})
df_available_offices = officesDataFrame(db,{'isAvailable':True})

#Take the 3 cities with highest number of tech startups
print('Number of startups in the 3 cities with most startups: ')
print(df_startups.city.value_counts().head(3))
print('Number of available offices for renting: ')
print(df_available_offices.city.value_counts().head(3))

Number of startups in the 3 cities with most startups: 
Cambridge        9
San Francisco    5
Redwood City     5
Name: city, dtype: int64
Number of available offices for renting: 
San Francisco    63
London           33
New York         33
Name: city, dtype: int64


In [16]:
#Keep only those offices in the desired cities
df_available_offices = df_available_offices[df_available_offices.
                                            apply(lambda s:s['city'] in selected_cities, axis = 1)]
df_startups = df_startups[df_startups.
                          apply(lambda s:s['city'] in selected_cities, axis = 1)]

db['startups'].insert_many(df_startups.T.to_dict().values())
db['rentings'].insert_many(df_available_offices.T.to_dict().values())